El presente script se ejecuta para un Enterprise. Lo primero que hay que hacer es obtener una capa, tipo poligono, de los municipios, y distritos de madrid. El nomre del distrito debe ser único. Junto con la fecha, serán los primary keys. 
Una vet tienes generada el featura, creas un campo, tipo caracter, que se llame 'plantilla', y le nombramos 1. Este es el punto cruciala, ya que esto permitirá elminiar, actualiza... sin tener que estar volviendo a cargar otra vez al capa.
dicionalmente, creamos un campo, tipo 'date' llamado fecha. 

Los campos creados, anteriormente citados, serán obligatorios para el funcionamiento. En mi cado he incluido un unico campo 'confirmados en los ultimos 14 días'. Peor se puede meter cualquiera que aparezca en el csv.


Respecto a los datos, los estoy cogiendo del opendata del ayuntamiento de Madrid. Se puede hacer el proceso análogo para otros datos, tan solo habría que modificar la ruta de datos csv, y el campo de datos.




In [4]:
from arcgis.gis import GIS
from datetime import datetime
import pandas as pd
from arcgis.features import FeatureLayerCollection
#parametros:
#csv--> datos actuales
rutaCovid='https://datos.comunidad.madrid/catalogo/dataset/7da43feb-8d4d-47e0-abd5-3d022d29d09e/resource/f22c3f43-c5d0-41a4-96dc-719214d56968/download/covid19_tia_muni_y_distritos_s.csv'
campoCSV1='casos_confirmados_ultimos_14dias'
campoCSVmuni='municipio_distrito'
campoCSVfecha='fecha_informe'
#feature-> capa:   donde se actualiza
idIntem_datos='0ee372de9faf421a8a04a09d0846f71f'
campoCapa1='confirmados14'
campoCapamuni='nombre'
campoCapafecha='fecha'



print("Inicio: "+datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
try:

    gis = GIS("home")
    #gis = GIS('SERVIDOR_ENTERPRISE/arcgis',USUARIO,PASS,verify_cert=False) 
except Exception as errorcito:
    print("Error al conectar con la cuenta de Arcgis - Enterprise ")
    print('Error: ' + str(errorcito))

#  leo datos csv:
try:
    print("Lectura csv")
    url=rutaCovid
    datos = pd.read_csv(url, encoding='latin-1', error_bad_lines=False, sep=';')
except Exception as errorcito:
    print("Error conectar con csv de la web ")
    print('Error: ' + str(errorcito))
#saco los distritos diferentes:
try:
    distritos=[]
    for i in datos[campoCSVmuni]:
        if i not in distritos:
            distritos.append(i)
except Exception as errorcito:
    print("Error al sacar los diferentes distritos ")
    print('Error: ' + str(errorcito))

#   leo los datos ya cargaddos para actualizarlos:
print("Lectura de datos de la capa actual")
try:
    print("Lectura de datos de la capa actual")
    datosOLD=gis.content.get(idIntem_datos)
    datos2 = FeatureLayerCollection.fromitem(datosOLD)
    capa=datos2.layers[0]
    print("Fin: "+datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
except Exception as errorcito:
    print("Error al conectar con la capa ya existente")
    print('Error: ' + str(errorcito))




Inicio: 2021/07/23 09:30:39
Lectura csv
Lectura de datos de la capa actual
Lectura de datos de la capa actual
Fin: 2021/07/23 09:30:44


In [5]:
try:
    #  limpio la capa
    print("Inicio: " + datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
    for uii in range(0, len(distritos)):

        jsonDEL = capa.query(campoCapamuni + "='" + distritos[uii] + "'" + "and plantilla is null or plantilla='0'",
                             out_fields=['*'])
        # print(campoCapamuni+"='"+distritos[uii]+"'"+ "and plantilla is null or plantilla='0'")
        if len(jsonDEL) > 0:
            capa.edit_features(deletes=jsonDEL)

    print("Capa limpiada")
    print("Fin: " + datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
except Exception as errorcito:
    print("Error al limpiar la  informacion de la capa")
    print('Error: ' + str(errorcito))

Inicio: 2021/07/23 09:31:03
Capa limpiada
Fin: 2021/07/23 09:32:49


In [7]:
# saco los datos del CSV
try:
    print("Inicio: " + datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
    csv_distrito = []
    csv_confirmados14 = []
    csv_fecha = []

    for ooo in range(0, len(datos[campoCSVmuni])):
        csv_distrito.append(datos[campoCSVmuni][ooo])
        csv_confirmados14.append(datos[campoCSV1][ooo])
        csv_fecha.append(datos[campoCSVfecha][ooo])

    print("saco la informacion del csv")
    print("Fin: " + datetime.now().strftime("%Y/%m/%d %H:%M:%S"))

except Exception as errorcito:
    print("Error sacar informacion del csv")
    print('Error: ' + str(errorcito))


Inicio: 2021/07/23 09:32:55
saco la informacion del csv
Fin: 2021/07/23 09:32:56


In [8]:
try:

    print("Inicio: " + datetime.now().strftime("%Y/%m/%d %H:%M:%S"))

    formatoFecha = "%Y/%m/%d %H:%M:%S"  # formato de fecha que lee del csv   #   probar:  '%d/%m/%y %H:%M:%S'
    # formatoFecha='%d/%m/%y %H:%M:%S'   #  formato de fecha que lee del csv   #   probar:  '%d/%m/%y %H:%M:%S'
    contador = 0
    for iii in range(0, len(distritos)):
        for jjj in range(0, len(csv_distrito)):
            if csv_distrito[jjj] == distritos[iii]:
                contador = contador + 1
                # jsonMETE=capa.query("nombre='"+distritos[iii]+"'"+"and plantilla='1'",out_fields=['*'])
                jsonMETE = capa.query(campoCapamuni + "='" + distritos[iii] + "'" + "and plantilla='1'", out_fields=['*'])
                jsonMETE.features[0].attributes[campoCapamuni] = distritos[iii]
                # jsonMETE.features[0].geometry=geometria[iii]
                jsonMETE.features[0].attributes['plantilla'] = '0'
                if str(csv_confirmados14[jjj]) == 'nan':
                    jsonMETE.features[0].attributes[campoCapa1] = None
                else:
                    jsonMETE.features[0].attributes[campoCapa1] = int(csv_confirmados14[jjj])
                date_time_obj = datetime.strptime(csv_fecha[jjj], formatoFecha)  # '%d/%m/%y %H:%M:%S')
                # jsonMETE.features[0].attributes['fecha']=csv_fecha[jjj]
                jsonMETE.features[0].attributes[campoCapafecha] = date_time_obj
                # 2021/06/29 12:04:00

                accion = capa.edit_features(adds=jsonMETE)
                if accion['addResults'][0].get('success') == False:
                    print("Error--> no se introduce el siguiente valor:")
                    print("Distrito: " + distritos[iii] + " fecha: " + csv_fecha[jjj] + "Confirmados: " + csv_confirmados14[
                        jjj])
                    print("Causa: " + accion['addResults'][0].get('error'))

    print("Meto los datos en la capa")
    datosOLD.update({'description' : "Ultima actualizacion "+datetime.now().strftime("%Y/%m/%d %H:%M:%S") +"-- Existen "+str(contador)+" datos"})
    print("Se han introducido " + str(contador) + " datos")
    print("Fin: " + datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
except Exception as errorcito:
    print("Error al introducir los datos en la capa")
    print('Error: ' + str(errorcito))



Inicio: 2021/07/23 09:33:05
Meto los datos en la capa
Se han introducido 11912 datos
Fin: 2021/07/23 09:48:52
